In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from scipy.special import erfc
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, VotingRegressor

from lightgbm import LGBMRegressor

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

import warnings
import datawig

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission=pd.read_csv('sample_submission.csv')

In [4]:
train.fillna('NaN', inplace=True) 
test.fillna('NaN', inplace=True)

바이너리 변수

In [52]:
# binary transform (0, 1)

train.gender = train.gender.replace(['F','M'],[0,1])  
test.gender = test.gender.replace(['F','M'],[0,1])

train.car = train.car.replace(['N','Y'],[0,1])  
test.car = test.car.replace(['N','Y'],[0,1])

train.reality = train.reality.replace(['N','Y'],[0,1])  
test.reality = test.reality.replace(['N','Y'],[0,1])

days변수 양수변환, 360나누기

In [53]:
# 음수 -> 양수 변환

train.DAYS_BIRTH = -train.DAYS_BIRTH
test.DAYS_BIRTH = -test.DAYS_BIRTH

train.DAYS_EMPLOYED = -train.DAYS_EMPLOYED
test.DAYS_EMPLOYED = -test.DAYS_EMPLOYED

train.begin_month = -train.begin_month
test.begin_month = -test.begin_month

In [54]:
# 365243 인 값 0으로 바꾸기

train.DAYS_EMPLOYED = train.DAYS_EMPLOYED.replace([-365243],[0])
test.DAYS_EMPLOYED = test.DAYS_EMPLOYED.replace([-365243],[0])

In [55]:
# 360으로 나눠서 나이, 일한 년수 계산

train['EMPLOYED']= train.DAYS_EMPLOYED.apply(lambda x: abs(x/360) )
train['age']= train.DAYS_BIRTH.apply(lambda x: abs(x/360) ).astype(int)
train['begin_month']= train.begin_month.apply(lambda x: abs(x) ).astype(int)

test['EMPLOYED']= test.DAYS_EMPLOYED.apply(lambda x: abs(x/360) )
test['age']= test.DAYS_BIRTH.apply(lambda x: abs(x/360) ).astype(int)
test['begin_month']= test.begin_month.apply(lambda x: abs(x) ).astype(int)

로그스케일

In [56]:
# train에 로그변환한 변수로 바꾸기
tr_it=train['income_total']
tr_it_log = np.log1p(tr_it)
train['income_total']= tr_it_log
# test set도
te_it=test['income_total']
te_it_log = np.log1p(te_it)
test['income_total']= te_it_log

In [57]:
# child_num이 14, 19인 행 지우기
idx_child_drop = train[(train['child_num'] == 14)|(train['child_num'] == 19)].index
train = train.drop(idx_child_drop)

In [58]:
# child_num이 family_size보다 큰 행도 지우기

idx_child_drop2 = train[train['family_size'] < train['child_num']].index
train = train.drop(idx_child_drop2)

파생변수

In [59]:
train['cf_ratio'] = train['child_num'] / train['family_size']
test['cf_ratio'] = test['child_num'] / test['family_size']

train['if_ratio'] = train['income_total'] / train['family_size']
test['if_ratio'] = test['income_total'] / test['family_size']

결측치처리

In [14]:
def unemployed(data):
    data.loc[(data['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'unemployed'

#train에 적용
unemployed(train)
#test에 적용
unemployed(test)

data wig

In [17]:
# data split
train_notnull = train[train.occyp_type.notnull()]
train_null = train[train.occyp_type.isnull()]

#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns=['EMPLOYED','age','income_type'], # column(s) containing information about the column we want to impute
    output_column='occyp_type', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

#Fit an imputer model on the train data
imputer.fit(train_df=train_notnull, num_epochs=50)

#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(train_null)

2021-05-30 17:08:48,699 [INFO]  CategoricalEncoder for column occyp_type                                found only 88 occurrences of value Secretaries
2021-05-30 17:08:48,702 [INFO]  CategoricalEncoder for column occyp_type                                found only 62 occurrences of value Realty agents
2021-05-30 17:08:48,702 [INFO]  CategoricalEncoder for column occyp_type                                found only 57 occurrences of value HR staff
2021-05-30 17:08:48,703 [INFO]  CategoricalEncoder for column occyp_type                                found only 38 occurrences of value IT staff
2021-05-30 17:08:49,263 [INFO]  
========== start: fit model
2021-05-30 17:08:49,264 [WARNING]  Already bound, ignoring bind()
2021-05-30 17:08:55,621 [INFO]  Epoch[0] Batch [0-639]	Speed: 1613.29 samples/sec	cross-entropy=1.737805	occyp_type-accuracy=0.406543
2021-05-30 17:09:02,210 [INFO]  Epoch[0] Train-cross-entropy=1.702629
2021-05-30 17:09:02,211 [INFO]  Epoch[0] Train-occyp_type-accuracy=0.

2021-05-30 17:11:44,945 [INFO]  Epoch[12] Validation-occyp_type-accuracy=0.430018
2021-05-30 17:11:51,499 [INFO]  Epoch[13] Batch [0-639]	Speed: 1562.69 samples/sec	cross-entropy=1.654103	occyp_type-accuracy=0.420215
2021-05-30 17:11:58,005 [INFO]  Epoch[13] Train-cross-entropy=1.644906
2021-05-30 17:11:58,006 [INFO]  Epoch[13] Train-occyp_type-accuracy=0.420335
2021-05-30 17:11:58,006 [INFO]  Epoch[13] Time cost=13.061
2021-05-30 17:11:58,011 [INFO]  Saved checkpoint to "imputer_model\model-0013.params"
2021-05-30 17:11:58,640 [INFO]  Epoch[13] Validation-cross-entropy=1.624163
2021-05-30 17:11:58,641 [INFO]  Epoch[13] Validation-occyp_type-accuracy=0.430018
2021-05-30 17:12:04,893 [INFO]  Epoch[14] Batch [0-639]	Speed: 1638.36 samples/sec	cross-entropy=1.653241	occyp_type-accuracy=0.420117
2021-05-30 17:12:11,054 [INFO]  Epoch[14] Train-cross-entropy=1.644068
2021-05-30 17:12:11,054 [INFO]  Epoch[14] Train-occyp_type-accuracy=0.420286
2021-05-30 17:12:11,055 [INFO]  Epoch[14] Time co

2021-05-30 17:14:51,651 [INFO]  Epoch[26] Validation-occyp_type-accuracy=0.430898
2021-05-30 17:14:58,328 [INFO]  Epoch[27] Batch [0-639]	Speed: 1534.19 samples/sec	cross-entropy=1.646025	occyp_type-accuracy=0.421777
2021-05-30 17:15:05,105 [INFO]  Epoch[27] Train-cross-entropy=1.637197
2021-05-30 17:15:05,106 [INFO]  Epoch[27] Train-occyp_type-accuracy=0.420921
2021-05-30 17:15:05,107 [INFO]  Epoch[27] Time cost=13.455
2021-05-30 17:15:05,112 [INFO]  Saved checkpoint to "imputer_model\model-0027.params"
2021-05-30 17:15:05,697 [INFO]  Epoch[27] Validation-cross-entropy=1.619518
2021-05-30 17:15:05,697 [INFO]  Epoch[27] Validation-occyp_type-accuracy=0.431338
2021-05-30 17:15:12,829 [INFO]  Epoch[28] Batch [0-639]	Speed: 1435.94 samples/sec	cross-entropy=1.645742	occyp_type-accuracy=0.422852
2021-05-30 17:15:19,661 [INFO]  Epoch[28] Train-cross-entropy=1.636869
2021-05-30 17:15:19,662 [INFO]  Epoch[28] Train-occyp_type-accuracy=0.421362
2021-05-30 17:15:19,663 [INFO]  Epoch[28] Time co

In [18]:
imputed = imputed.drop(['occyp_type','occyp_type_imputed_proba'],axis=1)

imputed = imputed.rename(columns = {'occyp_type_imputed':'occyp_type'})

train_imputed = pd.concat([train_notnull,imputed],axis = 0)

train_imputed.head()

,DAYS_BIRTH,DAYS_EMPLOYED,EMPLOYED,FLAG_MOBIL,age,begin_month,car,cf_ratio,child_num,credit,edu_type,email,family_size,family_type,gender,house_type,if_ratio,income_total,income_type,index,occyp_type,phone,reality,work_phone
1,11380,1540,4.277778,1,31,5,0,0.333333,1,1.0,Secondary / secondary special,1,3.0,Civil marriage,0,House / apartment,4.139723,12.419170,Commercial associate,1,Laborers,0,1,0
2,19087,4434,12.316667,1,53,22,1,0.000000,0,2.0,Higher education,0,2.0,Married,1,House / apartment,6.508503,13.017005,Working,2,Managers,1,1,0
3,15088,2092,5.811111,1,41,37,0,0.000000,0,0.0,Secondary / secondary special,0,2.0,Married,0,House / apartment,6.109250,12.218500,Commercial associate,3,Sales staff,1,1,0
4,15037,2105,5.847222,1,41,26,1,0.000000,0,2.0,Higher education,0,2.0,Married,0,House / apartment,5.983594,11.967187,State servant,4,Managers,0,1,0
5,13413,4996,13.877778,1,37,18,0,0.500000,2,1.0,Secondary / secondary special,1,4.0,Married,0,House / apartment,3.126545,12.506181,Working,5,High skill tech staff,0,1,0


In [19]:
# data split
test_notnull = test[test.occyp_type.notnull()]
test_null = test[test.occyp_type.isnull()]

#Initialize a SimpleImputer model
imputer = datawig.SimpleImputer(
    input_columns=['DAYS_EMPLOYED','DAYS_BIRTH','income_type'], # column(s) containing information about the column we want to impute
    output_column='occyp_type', # the column we'd like to impute values for
    output_path = 'imputer_model' # stores model data and metrics
    )

#Fit an imputer model on the train data
imputer.fit(train_df=test_notnull, num_epochs=50)

#Impute missing values and return original dataframe with predictions
imputed_2 = imputer.predict(test_null)

In [20]:
imputed_2 = imputed_2.drop(['occyp_type','occyp_type_imputed_proba'],axis=1)

imputed_2 = imputed_2.rename(columns = {'occyp_type_imputed':'occyp_type'})

test_imputed = pd.concat([test_notnull,imputed_2],axis = 0)

대체된 열 삭제

In [61]:
train = train.drop(['index', 'FLAG_MOBIL', 'DAYS_BIRTH', 'DAYS_EMPLOYED'],axis=1)
test = test.drop(['index', 'FLAG_MOBIL', 'DAYS_BIRTH', 'DAYS_EMPLOYED'],axis=1)

스케일링

In [62]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
numerical_feats.remove('credit')
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  14
Number of Categorical features:  5


In [63]:
encoder = OrdinalEncoder(categorical_feats)
train[categorical_feats] = encoder.fit_transform(train[categorical_feats], train['credit'])
test[categorical_feats] = encoder.transform(test[categorical_feats])

In [64]:
#numerical_feats.remove('income_total')
scaler = StandardScaler()
train[numerical_feats] = scaler.fit_transform(train[numerical_feats])
test[numerical_feats] = scaler.transform(test[numerical_feats])

모델링

In [71]:
n_est = 2000
seed = 42
n_fold = 7
n_class = 3

target = 'credit'
X = train.drop(target, axis=1)
y = train[target]
X_test = test

In [72]:
skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))
cat_cols = ['income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y.iloc[train_idx], y.iloc[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier()
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
  cat_pred_test += model_cat.predict_proba(X_test) / n_fold
  print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\tLog Loss: {log_loss(y, cat_pred):.6f}')



----------------- Fold 0 -----------------

Learning rate set to 0.114931
0:	learn: 1.0348127	test: 1.0335262	best: 1.0335262 (0)	total: 37.6ms	remaining: 37.6s
100:	learn: 0.7854994	test: 0.7940367	best: 0.7940367 (100)	total: 3.83s	remaining: 34.1s
200:	learn: 0.7540875	test: 0.7813367	best: 0.7813367 (200)	total: 7.8s	remaining: 31s
300:	learn: 0.7271243	test: 0.7725959	best: 0.7725834 (299)	total: 11.7s	remaining: 27.3s
400:	learn: 0.7036949	test: 0.7663053	best: 0.7663053 (400)	total: 15.7s	remaining: 23.5s
500:	learn: 0.6840680	test: 0.7630735	best: 0.7629294 (497)	total: 19.7s	remaining: 19.6s
600:	learn: 0.6650446	test: 0.7594588	best: 0.7594530 (596)	total: 23.5s	remaining: 15.6s
700:	learn: 0.6458350	test: 0.7561185	best: 0.7560757 (692)	total: 27.6s	remaining: 11.8s
800:	learn: 0.6281662	test: 0.7529388	best: 0.7528990 (798)	total: 31.5s	remaining: 7.83s
900:	learn: 0.6115147	test: 0.7514245	best: 0.7510631 (871)	total: 35.6s	remaining: 3.91s
999:	learn: 0.5968681	test: 0.7

CV Log Loss Score: 0.749768

----------------- Fold 1 -----------------

Learning rate set to 0.114931
0:	learn: 1.0342233	test: 1.0354075	best: 1.0354075 (0)	total: 32.8ms	remaining: 32.8s
100:	learn: 0.7882414	test: 0.8089793	best: 0.8089793 (100)	total: 3.47s	remaining: 30.9s
200:	learn: 0.7580928	test: 0.7970883	best: 0.7970883 (200)	total: 7.29s	remaining: 29s
300:	learn: 0.7307308	test: 0.7855386	best: 0.7855386 (300)	total: 11.2s	remaining: 26s
400:	learn: 0.7072605	test: 0.7770862	best: 0.7770862 (400)	total: 15.1s	remaining: 22.5s
500:	learn: 0.6868357	test: 0.7728152	best: 0.7728090 (499)	total: 19s	remaining: 18.9s
600:	learn: 0.6695504	test: 0.7691513	best: 0.7691193 (599)	total: 23s	remaining: 15.3s
700:	learn: 0.6511557	test: 0.7665864	best: 0.7665864 (700)	total: 26.9s	remaining: 11.5s
800:	learn: 0.6321025	test: 0.7624896	best: 0.7624625 (799)	total: 31.1s	remaining: 7.73s
900:	learn: 0.6165758	test: 0.7595673	best: 0.7594285 (897)	total: 35.2s	remaining: 3.87s
999:	lea

CV Log Loss Score: 0.756577

----------------- Fold 2 -----------------

Learning rate set to 0.114931
0:	learn: 1.0346939	test: 1.0340202	best: 1.0340202 (0)	total: 34.8ms	remaining: 34.7s
100:	learn: 0.7886206	test: 0.7986631	best: 0.7986631 (100)	total: 3.6s	remaining: 32.1s
200:	learn: 0.7533595	test: 0.7857098	best: 0.7857098 (200)	total: 7.61s	remaining: 30.3s
300:	learn: 0.7283973	test: 0.7791543	best: 0.7788511 (294)	total: 11.5s	remaining: 26.8s
400:	learn: 0.7043652	test: 0.7721638	best: 0.7721505 (388)	total: 15.4s	remaining: 23s
500:	learn: 0.6826658	test: 0.7665359	best: 0.7665359 (500)	total: 19.4s	remaining: 19.4s
600:	learn: 0.6611041	test: 0.7607745	best: 0.7606659 (593)	total: 23.5s	remaining: 15.6s
700:	learn: 0.6429710	test: 0.7563270	best: 0.7563270 (700)	total: 27.4s	remaining: 11.7s
800:	learn: 0.6261772	test: 0.7548931	best: 0.7548931 (800)	total: 31.6s	remaining: 7.84s
900:	learn: 0.6108433	test: 0.7524105	best: 0.7524105 (900)	total: 35.4s	remaining: 3.89s
999

CV Log Loss Score: 0.750665

----------------- Fold 3 -----------------

Learning rate set to 0.114931
0:	learn: 1.0343632	test: 1.0350264	best: 1.0350264 (0)	total: 36.7ms	remaining: 36.7s
100:	learn: 0.7858575	test: 0.8056297	best: 0.8056039 (99)	total: 3.66s	remaining: 32.6s
200:	learn: 0.7530668	test: 0.7923932	best: 0.7923932 (200)	total: 7.58s	remaining: 30.1s
300:	learn: 0.7268897	test: 0.7840186	best: 0.7840186 (300)	total: 11.4s	remaining: 26.4s
400:	learn: 0.7046676	test: 0.7788983	best: 0.7788983 (400)	total: 15.2s	remaining: 22.7s
500:	learn: 0.6830745	test: 0.7735113	best: 0.7735113 (500)	total: 19.1s	remaining: 19s
600:	learn: 0.6633399	test: 0.7690335	best: 0.7690335 (600)	total: 23s	remaining: 15.3s
700:	learn: 0.6440085	test: 0.7649072	best: 0.7649072 (700)	total: 27.6s	remaining: 11.8s
800:	learn: 0.6281079	test: 0.7634747	best: 0.7632094 (793)	total: 32.1s	remaining: 7.96s
900:	learn: 0.6116813	test: 0.7620252	best: 0.7618728 (897)	total: 36.5s	remaining: 4s
999:	lea

CV Log Loss Score: 0.760454

----------------- Fold 4 -----------------

Learning rate set to 0.114931
0:	learn: 1.0343657	test: 1.0350477	best: 1.0350477 (0)	total: 65ms	remaining: 1m 4s
100:	learn: 0.7875420	test: 0.8033870	best: 0.8033870 (100)	total: 3.88s	remaining: 34.5s
200:	learn: 0.7541067	test: 0.7919797	best: 0.7919797 (200)	total: 7.74s	remaining: 30.8s
300:	learn: 0.7287702	test: 0.7864391	best: 0.7863658 (291)	total: 11.8s	remaining: 27.5s
400:	learn: 0.7053386	test: 0.7805330	best: 0.7805059 (399)	total: 15.8s	remaining: 23.6s
500:	learn: 0.6842867	test: 0.7752364	best: 0.7752274 (499)	total: 19.8s	remaining: 19.8s
600:	learn: 0.6656097	test: 0.7702114	best: 0.7702114 (600)	total: 23.9s	remaining: 15.8s
700:	learn: 0.6465847	test: 0.7655063	best: 0.7655063 (700)	total: 27.8s	remaining: 11.9s
800:	learn: 0.6296857	test: 0.7625892	best: 0.7625892 (800)	total: 31.9s	remaining: 7.93s
900:	learn: 0.6131570	test: 0.7582599	best: 0.7581658 (897)	total: 35.9s	remaining: 3.95s
99

CV Log Loss Score: 0.756078

----------------- Fold 5 -----------------

Learning rate set to 0.114931
0:	learn: 1.0342025	test: 1.0354570	best: 1.0354570 (0)	total: 37ms	remaining: 36.9s
100:	learn: 0.7871297	test: 0.8065626	best: 0.8065626 (100)	total: 3.5s	remaining: 31.2s
200:	learn: 0.7516911	test: 0.7949742	best: 0.7949742 (200)	total: 7.6s	remaining: 30.2s
300:	learn: 0.7243437	test: 0.7866891	best: 0.7866891 (300)	total: 12.6s	remaining: 29.2s
400:	learn: 0.7016426	test: 0.7823816	best: 0.7822447 (398)	total: 17.7s	remaining: 26.4s
500:	learn: 0.6807025	test: 0.7765797	best: 0.7764928 (492)	total: 23.6s	remaining: 23.6s
600:	learn: 0.6616505	test: 0.7730765	best: 0.7730765 (600)	total: 29.8s	remaining: 19.8s
700:	learn: 0.6440421	test: 0.7694654	best: 0.7693677 (697)	total: 35s	remaining: 14.9s
800:	learn: 0.6255424	test: 0.7662179	best: 0.7662179 (800)	total: 39.5s	remaining: 9.8s
900:	learn: 0.6103168	test: 0.7653385	best: 0.7652564 (897)	total: 43.7s	remaining: 4.8s
999:	lea

CV Log Loss Score: 0.762481

----------------- Fold 6 -----------------

Learning rate set to 0.114931
0:	learn: 1.0350502	test: 1.0338725	best: 1.0338725 (0)	total: 36.8ms	remaining: 36.8s
100:	learn: 0.7880372	test: 0.7964744	best: 0.7964744 (100)	total: 3.88s	remaining: 34.6s
200:	learn: 0.7551577	test: 0.7832732	best: 0.7832732 (200)	total: 8.02s	remaining: 31.9s
300:	learn: 0.7295601	test: 0.7764884	best: 0.7764621 (299)	total: 12.3s	remaining: 28.6s
400:	learn: 0.7057455	test: 0.7707095	best: 0.7707095 (400)	total: 16.7s	remaining: 24.9s
500:	learn: 0.6853593	test: 0.7662313	best: 0.7662281 (493)	total: 22s	remaining: 21.9s
600:	learn: 0.6650273	test: 0.7623658	best: 0.7622959 (595)	total: 26.8s	remaining: 17.8s
700:	learn: 0.6445129	test: 0.7582366	best: 0.7582239 (699)	total: 31.6s	remaining: 13.5s
800:	learn: 0.6272979	test: 0.7551500	best: 0.7551500 (800)	total: 36.1s	remaining: 8.97s
900:	learn: 0.6109158	test: 0.7533118	best: 0.7531783 (889)	total: 40.9s	remaining: 4.49s
99

CV Log Loss Score: 0.750089
	Log Loss: 0.755159


In [ ]:
0.7565

In [73]:
submission.iloc[:, 1:] = cat_pred_test
submission

,index,0,1,2
0,26457,0.053179,0.132032,0.814788
1,26458,0.160528,0.134102,0.705369
2,26459,0.137893,0.126778,0.735329
3,26460,0.134668,0.141739,0.723593
4,26461,0.110908,0.215314,0.673779
...,...,...,...,...
9995,36452,0.146270,0.208863,0.644868
9996,36453,0.102100,0.294614,0.603286
9997,36454,0.023556,0.082668,0.893776
9998,36455,0.090996,0.155822,0.753182


In [74]:
submission.to_csv('sub_fin_2.csv', index=False)

0.73

스모트nc적용

In [75]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26452 entries, 0 to 26456
Data columns (total 19 columns):
gender          26452 non-null float64
car             26452 non-null float64
reality         26452 non-null float64
child_num       26452 non-null float64
income_total    26452 non-null float64
income_type     26452 non-null int32
edu_type        26452 non-null int32
family_type     26452 non-null int32
house_type      26452 non-null int32
work_phone      26452 non-null float64
phone           26452 non-null float64
email           26452 non-null float64
occyp_type      26452 non-null int32
family_size     26452 non-null float64
begin_month     26452 non-null float64
EMPLOYED        26452 non-null float64
age             26452 non-null float64
cf_ratio        26452 non-null float64
if_ratio        26452 non-null float64
dtypes: float64(14), int32(5)
memory usage: 3.5 MB


In [79]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import ADASYN # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import SMOTENC

In [80]:
print('Original dataset shape %s' % Counter(y))


#Create the oversampler. For SMOTE-NC we need to pinpoint the column position where is the categorical features are. In this case, 'IsActiveMember' is positioned in the second column we input [1] as the parameter. If you have more than one categorical columns, just input all the columns position
smote_nc = SMOTENC(categorical_features=[0,1,2,5,6,7,8,9,10,11,12], random_state=0)
X_over, y_over = smote_nc.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y_over))

Original dataset shape Counter({2.0: 16963, 1.0: 6267, 0.0: 3222})
Resampled dataset shape Counter({1.0: 16963, 2.0: 16963, 0.0: 16963})


In [81]:
skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X_over, y_over):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X_over.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))
cat_cols = ['income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X_over.iloc[train_idx], X_over.iloc[valid_idx], y_over.iloc[train_idx], y_over.iloc[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier()
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
  
  cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
  cat_pred_test += model_cat.predict_proba(X_test) / n_fold
  print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
    
print(f'\tLog Loss: {log_loss(y, cat_pred):.6f}')



----------------- Fold 0 -----------------

Learning rate set to 0.116445
0:	learn: 1.0871698	test: 1.0871864	best: 1.0871864 (0)	total: 55ms	remaining: 54.9s
100:	learn: 0.9157252	test: 0.9258042	best: 0.9258042 (100)	total: 5.79s	remaining: 51.6s
200:	learn: 0.8240499	test: 0.8416635	best: 0.8416635 (200)	total: 11.6s	remaining: 46.3s
300:	learn: 0.7643135	test: 0.7891895	best: 0.7891895 (300)	total: 17.3s	remaining: 40.3s
400:	learn: 0.7215758	test: 0.7520216	best: 0.7520216 (400)	total: 23.1s	remaining: 34.4s
500:	learn: 0.6847319	test: 0.7218169	best: 0.7218169 (500)	total: 28.9s	remaining: 28.8s
600:	learn: 0.6520672	test: 0.6959609	best: 0.6959609 (600)	total: 35.4s	remaining: 23.5s
700:	learn: 0.6257378	test: 0.6759504	best: 0.6759504 (700)	total: 42.7s	remaining: 18.2s
800:	learn: 0.6044768	test: 0.6610817	best: 0.6610817 (800)	total: 50s	remaining: 12.4s
900:	learn: 0.5861453	test: 0.6483492	best: 0.6483492 (900)	total: 56.3s	remaining: 6.19s
999:	learn: 0.5682476	test: 0.63

CV Log Loss Score: 0.636375

----------------- Fold 1 -----------------

Learning rate set to 0.116445
0:	learn: 1.0869234	test: 1.0871222	best: 1.0871222 (0)	total: 56.8ms	remaining: 56.7s
100:	learn: 0.9169797	test: 0.9243341	best: 0.9243341 (100)	total: 6.12s	remaining: 54.5s
200:	learn: 0.8253595	test: 0.8429232	best: 0.8429232 (200)	total: 12.5s	remaining: 49.6s
300:	learn: 0.7626831	test: 0.7866033	best: 0.7866033 (300)	total: 18.8s	remaining: 43.6s
400:	learn: 0.7171629	test: 0.7479894	best: 0.7479894 (400)	total: 25s	remaining: 37.4s
500:	learn: 0.6825438	test: 0.7208129	best: 0.7208129 (500)	total: 31.3s	remaining: 31.2s
600:	learn: 0.6541393	test: 0.6989678	best: 0.6989678 (600)	total: 37.5s	remaining: 24.9s
700:	learn: 0.6282010	test: 0.6795927	best: 0.6795927 (700)	total: 43.9s	remaining: 18.7s
800:	learn: 0.6064102	test: 0.6629724	best: 0.6629724 (800)	total: 50.4s	remaining: 12.5s
900:	learn: 0.5870750	test: 0.6494927	best: 0.6494927 (900)	total: 56.9s	remaining: 6.25s
99

CV Log Loss Score: 0.638187

----------------- Fold 2 -----------------

Learning rate set to 0.116445
0:	learn: 1.0874924	test: 1.0884841	best: 1.0884841 (0)	total: 66.4ms	remaining: 1m 6s
100:	learn: 0.9145869	test: 0.9346982	best: 0.9346982 (100)	total: 6.2s	remaining: 55.2s
200:	learn: 0.8210810	test: 0.8480166	best: 0.8480166 (200)	total: 12.8s	remaining: 50.8s
300:	learn: 0.7608091	test: 0.7956892	best: 0.7956892 (300)	total: 19.5s	remaining: 45.2s
400:	learn: 0.7127794	test: 0.7536255	best: 0.7536255 (400)	total: 25.9s	remaining: 38.7s
500:	learn: 0.6765812	test: 0.7245170	best: 0.7245170 (500)	total: 32.3s	remaining: 32.2s
600:	learn: 0.6475627	test: 0.7019087	best: 0.7019087 (600)	total: 39s	remaining: 25.9s
700:	learn: 0.6241332	test: 0.6840920	best: 0.6840920 (700)	total: 45.5s	remaining: 19.4s
800:	learn: 0.6022407	test: 0.6678655	best: 0.6678655 (800)	total: 52.1s	remaining: 12.9s
900:	learn: 0.5817164	test: 0.6527398	best: 0.6527398 (900)	total: 58.3s	remaining: 6.4s
999:

CV Log Loss Score: 0.642073

----------------- Fold 3 -----------------

Learning rate set to 0.116445
0:	learn: 1.0879097	test: 1.0879217	best: 1.0879217 (0)	total: 56.5ms	remaining: 56.4s
100:	learn: 0.9136641	test: 0.9301501	best: 0.9301501 (100)	total: 6.43s	remaining: 57.2s
200:	learn: 0.8241013	test: 0.8492282	best: 0.8492282 (200)	total: 12.6s	remaining: 50.1s
300:	learn: 0.7627191	test: 0.7953735	best: 0.7953735 (300)	total: 18.9s	remaining: 43.9s
400:	learn: 0.7157470	test: 0.7563485	best: 0.7563485 (400)	total: 26.3s	remaining: 39.3s
500:	learn: 0.6798921	test: 0.7277950	best: 0.7277950 (500)	total: 32.6s	remaining: 32.5s
600:	learn: 0.6510971	test: 0.7060494	best: 0.7060494 (600)	total: 39.2s	remaining: 26s
700:	learn: 0.6258415	test: 0.6873545	best: 0.6873545 (700)	total: 46s	remaining: 19.6s
800:	learn: 0.6044726	test: 0.6724532	best: 0.6724532 (800)	total: 52.4s	remaining: 13s
900:	learn: 0.5850893	test: 0.6590996	best: 0.6590996 (900)	total: 58.8s	remaining: 6.46s
999:	l

CV Log Loss Score: 0.647657

----------------- Fold 4 -----------------

Learning rate set to 0.116445
0:	learn: 1.0873933	test: 1.0868929	best: 1.0868929 (0)	total: 82.2ms	remaining: 1m 22s
100:	learn: 0.9118622	test: 0.9234808	best: 0.9234808 (100)	total: 6.44s	remaining: 57.3s
200:	learn: 0.8205960	test: 0.8417317	best: 0.8417317 (200)	total: 13s	remaining: 51.6s
300:	learn: 0.7632095	test: 0.7920508	best: 0.7920508 (300)	total: 19.3s	remaining: 44.8s
400:	learn: 0.7156174	test: 0.7525426	best: 0.7525426 (400)	total: 25.5s	remaining: 38.2s
500:	learn: 0.6816740	test: 0.7260407	best: 0.7260407 (500)	total: 31.9s	remaining: 31.8s
600:	learn: 0.6505611	test: 0.7025136	best: 0.7025136 (600)	total: 38.2s	remaining: 25.4s
700:	learn: 0.6218247	test: 0.6801272	best: 0.6801272 (700)	total: 44.7s	remaining: 19.1s
800:	learn: 0.5987157	test: 0.6640357	best: 0.6640357 (800)	total: 51.1s	remaining: 12.7s
900:	learn: 0.5782646	test: 0.6502996	best: 0.6502996 (900)	total: 57.6s	remaining: 6.33s
9

CV Log Loss Score: 0.640590

----------------- Fold 5 -----------------

Learning rate set to 0.116445
0:	learn: 1.0879272	test: 1.0880712	best: 1.0880712 (0)	total: 74.9ms	remaining: 1m 14s
100:	learn: 0.9181603	test: 0.9265148	best: 0.9265148 (100)	total: 6.91s	remaining: 1m 1s
200:	learn: 0.8236334	test: 0.8385121	best: 0.8385121 (200)	total: 13.6s	remaining: 54s
300:	learn: 0.7612258	test: 0.7808041	best: 0.7808041 (300)	total: 20.1s	remaining: 46.6s
400:	learn: 0.7198573	test: 0.7459612	best: 0.7459612 (400)	total: 26.2s	remaining: 39.1s
500:	learn: 0.6840828	test: 0.7156516	best: 0.7156516 (500)	total: 32.4s	remaining: 32.3s
600:	learn: 0.6547458	test: 0.6932378	best: 0.6932378 (600)	total: 39.1s	remaining: 25.9s
700:	learn: 0.6267616	test: 0.6712576	best: 0.6712576 (700)	total: 46.1s	remaining: 19.7s
800:	learn: 0.6037625	test: 0.6541324	best: 0.6541324 (800)	total: 53.5s	remaining: 13.3s
900:	learn: 0.5845562	test: 0.6412510	best: 0.6412510 (900)	total: 1m	remaining: 6.62s
999:

CV Log Loss Score: 0.627857

----------------- Fold 6 -----------------

Learning rate set to 0.116445
0:	learn: 1.0868858	test: 1.0870575	best: 1.0870575 (0)	total: 67.9ms	remaining: 1m 7s
100:	learn: 0.9216322	test: 0.9297417	best: 0.9297417 (100)	total: 6.75s	remaining: 1m
200:	learn: 0.8276638	test: 0.8456957	best: 0.8456957 (200)	total: 13.8s	remaining: 54.9s
300:	learn: 0.7654786	test: 0.7907657	best: 0.7907657 (300)	total: 21.3s	remaining: 49.6s
400:	learn: 0.7163698	test: 0.7498562	best: 0.7498562 (400)	total: 28.3s	remaining: 42.2s
500:	learn: 0.6815941	test: 0.7228575	best: 0.7228575 (500)	total: 34.7s	remaining: 34.6s
600:	learn: 0.6526202	test: 0.7005529	best: 0.7005529 (600)	total: 41.1s	remaining: 27.3s
700:	learn: 0.6271107	test: 0.6813612	best: 0.6813612 (700)	total: 47.6s	remaining: 20.3s
800:	learn: 0.6046733	test: 0.6652772	best: 0.6652772 (800)	total: 54.1s	remaining: 13.4s
900:	learn: 0.5843130	test: 0.6507618	best: 0.6507618 (900)	total: 1m	remaining: 6.66s
999:	l

CV Log Loss Score: 0.638030


ValueError: Found input variables with inconsistent numbers of samples: [50889, 26452]

In [82]:
submission=pd.read_csv('sample_submission.csv')
submission.iloc[:, 1:] = cat_pred_test
submission

,index,0,1,2
0,26457,0.042137,0.078919,0.878944
1,26458,0.242328,0.194570,0.563102
2,26459,0.289526,0.171335,0.539139
3,26460,0.179788,0.178418,0.641795
4,26461,0.127319,0.155446,0.717235
...,...,...,...,...
9995,36452,0.163822,0.210368,0.625810
9996,36453,0.159017,0.328554,0.512428
9997,36454,0.066337,0.129922,0.803740
9998,36455,0.187818,0.218333,0.593849


In [83]:
submission.to_csv('sub_fin_3.csv', index=False)